# Libraries

In [2]:
from __future__ import division
import dlib
from imutils import face_utils
import cv2
import numpy as np
from scipy.spatial import distance as dist
import threading
import pygame

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


# Alarm

In [3]:
def start_sound():
    pygame.mixer.init()
    pygame.mixer.music.load("z.ogg")
    pygame.mixer.music.play()

# Re-sampling using pixel area relation

In [4]:
def resize(img, width=None, height=None, interpolation=cv2.INTER_AREA):
    global ratio
    w, h = img.shape
    if width is None and height is None:
        return img
    elif width is None:
        ratio = height / h
        width = int(w * ratio)
        resized = cv2.resize(img, (height, width), interpolation)
        return resized
    else:
        ratio = width / w
        height = int(h * ratio)
        resized = cv2.resize(img, (height, width), interpolation)
        return resized

# Extracting the indexes of the given facial part and grabbing the (x, y)-coordinates from the shape NumPy array

In [5]:
def shape_to_np(shape, dtype="int"):
    coords = np.zeros((68, 2), dtype=dtype)
    for i in range(36, 48):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords

# Computing the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates

In [6]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C) # ear = eye

    # return the eye aspect ratio
    return ear

# Accessing camera & predictor files & converting the colour of the frame to gray & resizing it

In [7]:
camera = cv2.VideoCapture(0)

predictor_path = 'shape_predictor_68_face_landmarks.dat_2'

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
total = 0
alarm = False
while True:
    ret, frame = camera.read()
    if ret == False:
        print('Failed to capture frame from camera. Check camera index in cv2.VideoCapture(0) \n')
        break

    frame_grey = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_resized = resize(frame_grey, width=120)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(frame_resized, 1)

    if len(dets) > 0:
        for k, d in enumerate(dets):
            shape = predictor(frame_resized, d)
            shape = shape_to_np(shape)
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)
            ear = (leftEAR + rightEAR) / 2.0
            leftEyeHull = cv2.convexHull(leftEye)

            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
            if ear > .27:
                print(ear)
                total = 0
                alarm = False
                cv2.putText(frame, "Eyes Open ", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                total += 1
                if total > 20:
                    if not alarm:
                        alarm = True
                        d = threading.Thread(target=start_sound)
                        d.setDaemon(True)
                        d.start()
                        print("Stay Alert")
                        cv2.putText(frame, "drowsiness detected", (250, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 0, 0), 4)
                cv2.putText(frame, "Eyes close".format(total), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            for (x, y) in shape:
                cv2.circle(frame, (int(x / ratio), int(y / ratio)), 3, (255, 255, 255), -1)
    cv2.imshow("image", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        camera.release()
        break

0.39818413624
0.390703462927
0.285714285714
0.333333333333
0.285714285714
0.302576284107
0.294145284911
0.302576284107
0.302576284107
0.285714285714
0.304178319737
0.426417748642
0.399632034356
0.333026695297
0.297671356237
0.426417748642
0.385142353761
0.416197480144
0.426417748642
0.399632034356
0.41192907761
0.460201503534
0.424264068712
0.403112887415
0.428571428571
0.319992784809
0.302576284107
0.341175682602
0.317924188633
0.309523809524
0.297931526075
0.342543418237
0.406856891166
0.375
0.302606163161
0.300811429886
0.329759167567
0.47086305215
0.358674379449
0.354166666667
0.315647289112
0.353147289112
0.359782419688
0.371142605452
0.308131587119
0.27380952381
0.414398987305
0.366666666667
0.353005664792
0.380952380952
0.333333333333
0.31795480872
0.343169499062
0.366666666667
0.309523809524
0.464285714286
0.422619047619
0.331065653972
0.464285714286
0.357142857143
0.416666666667
0.309523809524
0.380952380952
0.355707070523
0.362938487863
0.362938487863
0.470081345006
0.3571428